In [19]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import sqlite3
import datetime
import boto3
import os
import tempfile



In [20]:
AWS_S3_BUCKET = os.getenv("S3_BUCKET")
AWS_ACCESS_KEY_ID = os.getenv("S3_KEY")
AWS_SECRET_ACCESS_KEY = os.getenv("S3_SECRET")

s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY
)


Credit to Stockanalysis.com for providing this public data to us

In [21]:
page  = requests.get("https://stockanalysis.com/list/biggest-companies/")
page.content


b'<!DOCTYPE html>\n<html lang="en">\n\t<head>\n\t\t<meta charset="utf-8" />\n\t\t<meta name="viewport" content="width=device-width, initial-scale=1" />\n\t\t\n\t\t<link href="../../_app/immutable/assets/0.c55435d4.css" rel="stylesheet">\n\t\t<link href="../../_app/immutable/assets/___StockTable.f29087bf.css" rel="stylesheet"><title>All U.S. Listed Stocks, Ranked by Market Cap - Stock Analysis</title><!-- HEAD_svelte-1ipx2c_START --><meta name="description" content="A list of all stocks listed on the NASDAQ, NYSE and NYSE American in the United States, sorted by market cap and updated daily."><meta name="robots" content="index, follow"><link rel="canonical" href="https://stockanalysis.com/list/biggest-companies/"><meta property="og:title" content="All U.S. Listed Stocks, Ranked by Market Cap - Stock Analysis"><meta property="og:description" content="A list of all stocks listed on the NASDAQ, NYSE and NYSE American in the United States, sorted by market cap and updated daily."><meta prop

Scrape the HTML we need from the page and use the prettify method to organize it

In [22]:
soup = BeautifulSoup(page.content, 'html.parser')
print(soup.prettify())


<!DOCTYPE html>
<html lang="en">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <link href="../../_app/immutable/assets/0.c55435d4.css" rel="stylesheet"/>
  <link href="../../_app/immutable/assets/___StockTable.f29087bf.css" rel="stylesheet"/>
  <title>
   All U.S. Listed Stocks, Ranked by Market Cap - Stock Analysis
  </title>
  <!-- HEAD_svelte-1ipx2c_START -->
  <meta content="A list of all stocks listed on the NASDAQ, NYSE and NYSE American in the United States, sorted by market cap and updated daily." name="description"/>
  <meta content="index, follow" name="robots"/>
  <link href="https://stockanalysis.com/list/biggest-companies/" rel="canonical"/>
  <meta content="All U.S. Listed Stocks, Ranked by Market Cap - Stock Analysis" property="og:title"/>
  <meta content="A list of all stocks listed on the NASDAQ, NYSE and NYSE American in the United States, sorted by market cap and updated daily." property="og:description"/>

Now it's just a matter of turning the data into the columns we need

In [23]:
stocks = soup.find(class_='svelte-1tv1ofl')
stocks

<div class="styled-scroll overflow-x-auto svelte-1tv1ofl"><table class="symbol-table svelte-1tv1ofl" id="main-table"><thead class="svelte-1tv1ofl"><tr class="svelte-1tv1ofl"><th class="svelte-1tv1ofl" draggable="true" id="no">No.
					</th><th class="sym svelte-1tv1ofl" draggable="true" id="s">Symbol
					</th><th class="slw svelte-1tv1ofl" draggable="true" id="n">Company Name
					</th><th class="svelte-1tv1ofl" draggable="true" id="marketCap">
<span class="svelte-1tv1ofl">Market Cap <!-- HTML_TAG_START --><svg class="mt-0.5 w-4 h-4" fill="currentColor" style="max-width:40px" viewbox="0 0 20 20"><path clip-rule="evenodd" d="M5.293 7.293a1 1 0 011.414 0L10 10.586l3.293-3.293a1 1 0 111.414 1.414l-4 4a1 1 0 01-1.414 0l-4-4a1 1 0 010-1.414z" fill-rule="evenodd"></path></svg><!-- HTML_TAG_END --></span>
</th><th class="svelte-1tv1ofl" draggable="true" id="price">Stock Price
					</th><th class="svelte-1tv1ofl" draggable="true" id="change">% Change
					</th><th class="tr svelte-1tv1ofl" dr

In [24]:
names = stocks.find_all(class_='slw svelte-1tv1ofl')
names

[<th class="slw svelte-1tv1ofl" draggable="true" id="n">Company Name
 					</th>,
 <td class="slw svelte-1tv1ofl">Apple Inc.</td>,
 <td class="slw svelte-1tv1ofl">Microsoft Corporation</td>,
 <td class="slw svelte-1tv1ofl">Alphabet Inc.</td>,
 <td class="slw svelte-1tv1ofl">Amazon.com, Inc.</td>,
 <td class="slw svelte-1tv1ofl">NVIDIA Corporation</td>,
 <td class="slw svelte-1tv1ofl">Tesla, Inc.</td>,
 <td class="slw svelte-1tv1ofl">Berkshire Hathaway Inc.</td>,
 <td class="slw svelte-1tv1ofl">Meta Platforms, Inc.</td>,
 <td class="slw svelte-1tv1ofl">Taiwan Semiconductor Manufacturing Company Limited</td>,
 <td class="slw svelte-1tv1ofl">Visa Inc.</td>,
 <td class="slw svelte-1tv1ofl">UnitedHealth Group Incorporated</td>,
 <td class="slw svelte-1tv1ofl">Exxon Mobil Corporation</td>,
 <td class="slw svelte-1tv1ofl">Eli Lilly and Company</td>,
 <td class="slw svelte-1tv1ofl">Johnson &amp; Johnson</td>,
 <td class="slw svelte-1tv1ofl">JPMorgan Chase &amp; Co.</td>,
 <td class="slw svelte

In [25]:
symbols = stocks.find_all(class_='sym svelte-1tv1ofl')
symbols

[<th class="sym svelte-1tv1ofl" draggable="true" id="s">Symbol
 					</th>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/aapl/">AAPL</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/msft/">MSFT</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/googl/">GOOGL</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/amzn/">AMZN</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/nvda/">NVDA</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/tsla/">TSLA</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/brk.b/">BRK.B</a><!-- HTML_TAG_END --></td>,
 <td class="sym svelte-1tv1ofl"><!-- HTML_TAG_START --><a href="/stocks/meta/">META</a><!-- HTML_TAG_END --></td>,


In [26]:
prices = stocks.find_all(class_="svelte-1tv1ofl")
prices

[<table class="symbol-table svelte-1tv1ofl" id="main-table"><thead class="svelte-1tv1ofl"><tr class="svelte-1tv1ofl"><th class="svelte-1tv1ofl" draggable="true" id="no">No.
 					</th><th class="sym svelte-1tv1ofl" draggable="true" id="s">Symbol
 					</th><th class="slw svelte-1tv1ofl" draggable="true" id="n">Company Name
 					</th><th class="svelte-1tv1ofl" draggable="true" id="marketCap">
 <span class="svelte-1tv1ofl">Market Cap <!-- HTML_TAG_START --><svg class="mt-0.5 w-4 h-4" fill="currentColor" style="max-width:40px" viewbox="0 0 20 20"><path clip-rule="evenodd" d="M5.293 7.293a1 1 0 011.414 0L10 10.586l3.293-3.293a1 1 0 111.414 1.414l-4 4a1 1 0 01-1.414 0l-4-4a1 1 0 010-1.414z" fill-rule="evenodd"></path></svg><!-- HTML_TAG_END --></span>
 </th><th class="svelte-1tv1ofl" draggable="true" id="price">Stock Price
 					</th><th class="svelte-1tv1ofl" draggable="true" id="change">% Change
 					</th><th class="tr svelte-1tv1ofl" draggable="true" id="revenue">Revenue
 					</th></t

In [27]:
stock_names = []
for stock in names:
    stock_names.append(stock.get_text())

stock_names

['Company Name\n\t\t\t\t\t',
 'Apple Inc.',
 'Microsoft Corporation',
 'Alphabet Inc.',
 'Amazon.com, Inc.',
 'NVIDIA Corporation',
 'Tesla, Inc.',
 'Berkshire Hathaway Inc.',
 'Meta Platforms, Inc.',
 'Taiwan Semiconductor Manufacturing Company Limited',
 'Visa Inc.',
 'UnitedHealth Group Incorporated',
 'Exxon Mobil Corporation',
 'Eli Lilly and Company',
 'Johnson & Johnson',
 'JPMorgan Chase & Co.',
 'Walmart Inc.',
 'Novo Nordisk A/S',
 'Mastercard Incorporated',
 'The Procter & Gamble Company',
 'Broadcom Inc.',
 'Chevron Corporation',
 'The Home Depot, Inc.',
 'Oracle Corporation',
 'ASML Holding N.V.',
 'Merck & Co., Inc.',
 'The Coca-Cola Company',
 'PepsiCo, Inc.',
 'AbbVie Inc.',
 'Bank of America Corporation',
 'Costco Wholesale Corporation',
 'AstraZeneca PLC',
 'Pfizer Inc.',
 'Alibaba Group Holding Limited',
 'Novartis AG',
 'Adobe Inc.',
 'BHP Group Limited',
 'Salesforce, Inc.',
 "McDonald's Corporation",
 'Cisco Systems, Inc.',
 'Advanced Micro Devices, Inc.',
 'Toyot

In [28]:
stock_symbols = []
for stock in symbols:
    stock_symbols.append(stock.get_text())

stock_symbols

['Symbol\n\t\t\t\t\t',
 'AAPL',
 'MSFT',
 'GOOGL',
 'AMZN',
 'NVDA',
 'TSLA',
 'BRK.B',
 'META',
 'TSM',
 'V',
 'UNH',
 'XOM',
 'LLY',
 'JNJ',
 'JPM',
 'WMT',
 'NVO',
 'MA',
 'PG',
 'AVGO',
 'CVX',
 'HD',
 'ORCL',
 'ASML',
 'MRK',
 'KO',
 'PEP',
 'ABBV',
 'BAC',
 'COST',
 'AZN',
 'PFE',
 'BABA',
 'NVS',
 'ADBE',
 'BHP',
 'CRM',
 'MCD',
 'CSCO',
 'AMD',
 'TM',
 'TMO',
 'SHEL',
 'ACN',
 'FMX',
 'NFLX',
 'LIN',
 'ABT',
 'DHR',
 'DIS',
 'CMCSA',
 'NKE',
 'WFC',
 'TMUS',
 'TXN',
 'SAP',
 'HSBC',
 'NEE',
 'VZ',
 'RTX',
 'MS',
 'TTE',
 'PM',
 'UPS',
 'BMY',
 'QCOM',
 'HON',
 'BA',
 'INTC',
 'RY',
 'SNY',
 'AXP',
 'COP',
 'UL',
 'LOW',
 'IBM',
 'SPGI',
 'UNP',
 'SONY',
 'HDB',
 'CAT',
 'INTU',
 'AMGN',
 'LMT',
 'NOW',
 'T',
 'AMAT',
 'GE',
 'PLD',
 'SBUX',
 'ELV',
 'DE',
 'GS',
 'MDT',
 'BUD',
 'TD',
 'ISRG',
 'BX',
 'RIO',
 'SYK',
 'BP',
 'BLK',
 'PDD',
 'MDLZ',
 'BKNG',
 'SCHW',
 'GILD',
 'DEO',
 'C',
 'CVS',
 'ADI',
 'TJX',
 'EQNR',
 'ADP',
 'AMT',
 'PBR',
 'MMC',
 'VRTX',
 'SHOP',
 'MUFG',

In [29]:
count = 0
count2 = 0
stock_prices = []
for stock in prices:
    if count >= 13:
        stock_prices.append(stock.get_text())
    count += 1

stock_prices2 = []
for stock in stock_prices:
    if count2 == 4:
        stock_prices2.append(stock)
    count2 += 1
    if count2 > 7:
        count2 = 0

stock_prices2

['181.52',
 '328.06',
 '122.92',
 '124.35',
 '392.30',
 '248.47',
 '336.54',
 '265.15',
 '103.11',
 '224.22',
 '495.56',
 '107.99',
 '445.02',
 '160.39',
 '141.19',
 '152.62',
 '157.48',
 '370.00',
 '145.99',
 '813.26',
 '159.83',
 '298.04',
 '109.76',
 '716.91',
 '111.13',
 '60.59',
 '182.44',
 '138.60',
 '29.26',
 '518.85',
 '73.96',
 '39.43',
 '85.43',
 '100.14',
 '461.33',
 '60.05',
 '215.93',
 '286.20',
 '49.85',
 '125.54',
 '148.41',
 '519.44',
 '58.63',
 '309.71',
 '106.73',
 '416.23',
 '360.18',
 '101.02',
 '235.21',
 '92.17',
 '40.34',
 '105.67',
 '42.34',
 '130.66',
 '171.75',
 '132.28',
 '38.26',
 '74.60',
 '35.54',
 '99.27',
 '86.65',
 '59.22',
 '92.32',
 '170.50',
 '65.23',
 '119.01',
 '197.71',
 '217.42',
 '30.99',
 '92.75',
 '51.17',
 '171.60',
 '104.94',
 '49.82',
 '210.25',
 '135.68',
 '381.70',
 '200.25',
 '98.72',
 '65.05',
 '233.91',
 '425.52',
 '220.79',
 '463.81',
 '538.47',
 '16.09',
 '136.84',
 '105.38',
 '123.07',
 '98.21',
 '474.48',
 '380.66',
 '333.73',
 '83

In [30]:
count3 = 0
market_cap = []
for stock in stock_prices:
    if count3 == 3:
        market_cap.append(stock)
    count3 += 1
    if count3 > 7:
        count3 = 0

market_cap

['2,854.13B',
 '2,441.15B',
 '1,564.91B',
 '1,276.50B',
 '968.05B',
 '789.84B',
 '737.05B',
 '679.69B',
 '534.82B',
 '475.69B',
 '461.37B',
 '436.24B',
 '422.29B',
 '416.78B',
 '412.48B',
 '411.01B',
 '353.33B',
 '347.73B',
 '343.75B',
 '338.86B',
 '302.73B',
 '299.64B',
 '296.25B',
 '283.09B',
 '281.86B',
 '261.87B',
 '251.21B',
 '244.49B',
 '233.10B',
 '230.00B',
 '229.24B',
 '222.62B',
 '219.12B',
 '212.19B',
 '211.61B',
 '211.13B',
 '210.42B',
 '208.94B',
 '203.21B',
 '201.97B',
 '201.36B',
 '200.36B',
 '199.95B',
 '195.66B',
 '191.02B',
 '185.25B',
 '176.33B',
 '175.69B',
 '173.52B',
 '168.40B',
 '167.81B',
 '162.38B',
 '158.83B',
 '156.79B',
 '156.03B',
 '154.39B',
 '152.32B',
 '150.97B',
 '149.41B',
 '145.06B',
 '144.70B',
 '144.19B',
 '143.28B',
 '143.16B',
 '137.05B',
 '132.76B',
 '131.62B',
 '130.73B',
 '129.35B',
 '128.88B',
 '127.78B',
 '127.51B',
 '127.01B',
 '125.35B',
 '123.23B',
 '123.16B',
 '122.45B',
 '122.08B',
 '121.83B',
 '121.25B',
 '120.77B',
 '119.23B',
 '117.92

In [31]:
count4 = 0
percent_change = []
for stock in stock_prices:
    if count4 == 5:
        percent_change.append(stock)
    count4 += 1
    if count4 > 7:
        count4 = 0

percent_change

['0.53%',
 '0.86%',
 '0.64%',
 '0.08%',
 '1.87%',
 '5.79%',
 '0.18%',
 '0.22%',
 '3.17%',
 '0.52%',
 '1.09%',
 '-0.18%',
 '-0.14%',
 '0.08%',
 '0.33%',
 '0.30%',
 '-1.24%',
 '0.66%',
 '-0.31%',
 '1.13%',
 '0.37%',
 '-1.01%',
 '2.12%',
 '-0.44%',
 '0.73%',
 '0.36%',
 '0.19%',
 '0.70%',
 '0.09%',
 '-0.24%',
 '-0.39%',
 '0.87%',
 '-0.82%',
 '-0.51%',
 '5.08%',
 '0.06%',
 '3.05%',
 '0.15%',
 '0.34%',
 '3.71%',
 '1.06%',
 '0.81%',
 '-0.37%',
 '0.77%',
 '1.79%',
 '1.68%',
 '-0.56%',
 '0.24%',
 '-0.08%',
 '-0.39%',
 '-0.32%',
 '-0.50%',
 '-0.07%',
 '0.26%',
 '-0.28%',
 '-0.70%',
 '-0.49%',
 '-0.40%',
 '0.31%',
 '0.19%',
 '0.39%',
 '-0.45%',
 '0.16%',
 '-0.57%',
 '-0.12%',
 '2.29%',
 '-0.22%',
 '-0.32%',
 '-2.62%',
 '0.44%',
 '-0.72%',
 '0.27%',
 '0.38%',
 '-0.71%',
 '-0.10%',
 '0.94%',
 '-0.12%',
 '-0.37%',
 '1.34%',
 '0.57%',
 '-0.23%',
 '0.51%',
 '0.01%',
 '-0.08%',
 '0.58%',
 '0.09%',
 '0.47%',
 '-0.65%',
 '-1.47%',
 '-0.95%',
 '1.76%',
 '-0.03%',
 '-0.52%',
 '0.04%',
 '0.15%',
 '0.68%',
 

In [32]:
count5 = 0
revenue = []
for stock in stock_prices:
    if count5 == 6:
        revenue.append(stock)
    count5 += 1
    if count5 > 7:
        count5 = 0

revenue

['385.10B',
 '207.59B',
 '284.61B',
 '524.90B',
 '25.88B',
 '86.04B',
 '353.41B',
 '117.35B',
 '74.80B',
 '30.98B',
 '335.94B',
 '409.74B',
 '27.69B',
 '96.26B',
 '136.33B',
 '622.02B',
 '27.57B',
 '22.82B',
 '80.97B',
 '35.04B',
 '242.67B',
 '155.75B',
 '47.96B',
 '26.51B',
 '57.87B',
 '43.49B',
 '88.04B',
 '56.74B',
 '97.98B',
 '235.44B',
 '43.84B',
 '92.95B',
 '126.45B',
 '52.22B',
 '18.00B',
 '60.28B',
 '32.19B',
 '23.41B',
 '54.90B',
 '23.07B',
 '280.62B',
 '43.81B',
 '386.20B',
 '63.14B',
 '34.56B',
 '31.91B',
 '33.35B',
 '41.51B',
 '30.95B',
 '86.98B',
 '120.11B',
 '50.63B',
 '76.92B',
 '79.08B',
 '19.50B',
 '33.67B',
 '61.60B',
 '24.78B',
 '136.19B',
 '68.57B',
 '53.38B',
 '385.49B',
 '80.65B',
 '98.89B',
 '45.85B',
 '41.07B',
 '35.95B',
 '70.54B',
 '56.42B',
 '39.79B',
 '48.29B',
 '52.14B',
 '78.38B',
 '63.91B',
 '95.75B',
 '60.58B',
 '11.95B',
 '25.07B',
 '87.16B',
 '11.71B',
 '61.70B',
 '14.07B',
 '26.19B',
 '66.15B',
 '7.62B',
 '121.17B',
 '26.64B',
 '74.00B',
 '6.52B',
 '3

In [33]:
data = {
    'Name': stock_names[1 : 301],
    'Symbol': stock_symbols[1 : 301],
    'Price': stock_prices2[0 : 300],
    'Market_Cap': market_cap[0 : 300],
    'Percent_Change': percent_change[0 : 300],
    'Revenue': revenue[0 : 300],
}

df = pd.DataFrame(data)
df['Timestamp'] = datetime.datetime.utcnow()
df = df.reset_index()
df = df.rename(columns={'index' : 'id'})
df


,id,Name,Symbol,Price,Market_Cap,Percent_Change,Revenue,Timestamp
0,0,Apple Inc.,AAPL,181.52,"2,854.13B",0.53%,385.10B,2023-06-09 14:47:14.501162
1,1,Microsoft Corporation,MSFT,328.06,"2,441.15B",0.86%,207.59B,2023-06-09 14:47:14.501162
2,2,Alphabet Inc.,GOOGL,122.92,"1,564.91B",0.64%,284.61B,2023-06-09 14:47:14.501162
3,3,"Amazon.com, Inc.",AMZN,124.35,"1,276.50B",0.08%,524.90B,2023-06-09 14:47:14.501162
4,4,NVIDIA Corporation,NVDA,392.30,968.05B,1.87%,25.88B,2023-06-09 14:47:14.501162
...,...,...,...,...,...,...,...,...
295,295,"The Williams Companies, Inc.",WMB,30.92,37.67B,-2.46%,11.52B,2023-06-09 14:47:14.501162
296,296,Centene Corporation,CNC,68.45,37.54B,1.23%,146.25B,2023-06-09 14:47:14.501162
297,297,Carrier Global Corporation,CARR,44.95,37.53B,-1.43%,21.04B,2023-06-09 14:47:14.501162
298,298,"IDEXX Laboratories, Inc.",IDXX,451.73,37.50B,-0.28%,3.43B,2023-06-09 14:47:14.501162


In [34]:
df.head(5)
df.dtypes
df['Name'] = df['Name'].astype('string')
df['Symbol'] = df['Symbol'].astype('string')
df.replace(',','', regex=True, inplace=True)
df['Price'] = df['Price'].astype(float)
perc_change = df['Percent_Change'].replace('%','', regex=True, inplace=True)
df

,id,Name,Symbol,Price,Market_Cap,Percent_Change,Revenue,Timestamp
0,0,Apple Inc.,AAPL,181.52,2854.13B,0.53,385.10B,2023-06-09 14:47:14.501162
1,1,Microsoft Corporation,MSFT,328.06,2441.15B,0.86,207.59B,2023-06-09 14:47:14.501162
2,2,Alphabet Inc.,GOOGL,122.92,1564.91B,0.64,284.61B,2023-06-09 14:47:14.501162
3,3,Amazon.com Inc.,AMZN,124.35,1276.50B,0.08,524.90B,2023-06-09 14:47:14.501162
4,4,NVIDIA Corporation,NVDA,392.30,968.05B,1.87,25.88B,2023-06-09 14:47:14.501162
...,...,...,...,...,...,...,...,...
295,295,The Williams Companies Inc.,WMB,30.92,37.67B,-2.46,11.52B,2023-06-09 14:47:14.501162
296,296,Centene Corporation,CNC,68.45,37.54B,1.23,146.25B,2023-06-09 14:47:14.501162
297,297,Carrier Global Corporation,CARR,44.95,37.53B,-1.43,21.04B,2023-06-09 14:47:14.501162
298,298,IDEXX Laboratories Inc.,IDXX,451.73,37.50B,-0.28,3.43B,2023-06-09 14:47:14.501162


In [35]:
db = sqlite3.connect('../instance/dev.db')

df.to_sql('stocks', con=db, if_exists='replace', schema='stocks', index=False)


300

In [36]:
tf = tempfile.NamedTemporaryFile()

s3_client.put_object(
    ACL='public-read',
    Bucket=AWS_S3_BUCKET,
    Body=df.to_csv(),
    Key=tf.name
)

{'ResponseMetadata': {'RequestId': '5QRTVC3EMHJTYXGF',
  'HostId': 'gP6qFuUoxIZmOa2Qgjqp3HLPrDzVb5qu2+V9PaqjOI7FyJimuAyc6OX4k7Uepoyg/3SeffT0yBZZqYuIiOI4VnQ/W3fKPC7mmoULYuJ3DiM=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'gP6qFuUoxIZmOa2Qgjqp3HLPrDzVb5qu2+V9PaqjOI7FyJimuAyc6OX4k7Uepoyg/3SeffT0yBZZqYuIiOI4VnQ/W3fKPC7mmoULYuJ3DiM=',
   'x-amz-request-id': '5QRTVC3EMHJTYXGF',
   'date': 'Fri, 09 Jun 2023 14:47:16 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"dcb7bead70517efddbc2195b9fbb0f36"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"dcb7bead70517efddbc2195b9fbb0f36"',
 'ServerSideEncryption': 'AES256'}